In [7]:
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from scripts.utils import cross_validation, preprocessing

In [8]:
f = lambda time: Timestamp.fromtimestamp(time*3600)
f2 = np.vectorize(f)

In [9]:
ids = ['aq_54', 'aq_76', 'aq_94','aq_g507','aq_g508','aq_g509', 'aq_g512', 'aq_91', 'ANQ16PZJ', 'ALS2LCWY', 
       'AB6051M4', 'AW66FF7V', 'A743BPWK']

In [12]:
filename = '../data/final_data.csv'
data = pd.read_csv(filename, parse_dates=['time'], usecols =['time', 'latitude', 'longitude', 'pm2_5'])
data.head()

,time,latitude,longitude,pm2_5
0,2021-06-22 12:50:09+00:00,0.3564,32.573,16.62
1,2021-06-22 12:44:38+00:00,0.3564,32.573,23.10
2,2021-06-22 12:40:36+00:00,0.3564,32.573,18.48
3,2021-06-22 12:35:09+00:00,0.3564,32.573,15.77
4,2021-06-22 12:30:57+00:00,0.3564,32.573,29.62


In [ ]:
final_df = pd.DataFrame()
start = pd.Timestamp('2021-06-15 00:00:00', tz='UTC')
end = pd.Timestamp('2021-06-21 23:59:59', tz='UTC')
for i, latitude in enumerate(latitudes):
    df = data[data['latitude'] == latitude]
    longitude = longitudes[i]
    processed_df = preprocessing(df, latitude, longitude, start, end)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()